<a href="https://colab.research.google.com/github/daniivelascoo/ifp-programacion-ia/blob/main/Lab_1_3_Spotify_Pandas_Student.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🎵 Laboratorio 1.3: Entrenamiento Pandas (Spotify)
**Núcleo Formativo 1 - Ingeniería de Datos**

---
### 📜 Contexto
Antes de enfrentarte al Proyecto Final del Núcleo 1, necesitamos poner a punto tus habilidades de **Data Wrangling**.

Vamos a trabajar con datos simulados de Spotify. Tenemos dos tablas:
1.  **Tracks:** Canciones con sus métricas técnicas (Popularidad, Duración).
2.  **Artists:** Información de los creadores (Género, Seguidores).

### 🎯 Tu Misión
Realizar las 4 operaciones fundamentales de Pandas:
1.  **Limpieza:** Arreglar nulos y errores.
2.  **Fusión:** Unir tablas (Merge).
3.  **Agregación:** Calcular estadísticas por grupo.
4.  **Transformación:** Crear categorías con lógica personalizada.

---

In [1]:
# --- ⚙️ 0. GENERACIÓN DE DATOS (NO TOCAR) ---
# Ejecuta esta celda para crear los datos en la memoria de Colab.
import pandas as pd
import numpy as np

# Semilla para que los datos sean siempre iguales
np.random.seed(42)

def crear_datos_spotify():
    # 1. Tabla ARTISTAS
    n_artists = 20
    generos = ['Pop', 'Rock', 'Hip-Hop', 'Latino', 'Jazz']
    df_artistas = pd.DataFrame({
        'artist_id': [f'ART_{i:02d}' for i in range(n_artists)],
        'name': [f'Artista_{i}' for i in range(n_artists)],
        'genre': np.random.choice(generos, n_artists),
        'followers': np.random.randint(1000, 1000000, n_artists)
    })

    # 2. Tabla CANCIONES (Tracks)
    n_tracks = 100
    df_tracks = pd.DataFrame({
        'track_id': [f'TRK_{i:03d}' for i in range(n_tracks)],
        'title': [f'Cancion_{i}' for i in range(n_tracks)],
        'artist_id': np.random.choice(df_artistas['artist_id'], n_tracks),
        'popularity': np.random.randint(0, 100, n_tracks).astype(float),
        'duration_ms': np.random.randint(120000, 300000, n_tracks)
    })

    # Introducir errores
    # Nulos en popularidad
    df_tracks.loc[df_tracks.sample(5).index, 'popularity'] = np.nan
    # Duraciones negativas
    df_tracks.loc[df_tracks.sample(3).index, 'duration_ms'] = -50000

    return df_tracks, df_artistas

tracks, artists = crear_datos_spotify()

print("✅ Datos cargados correctamente.")
print(f"   - Tracks: {tracks.shape}")
print(f"   - Artists: {artists.shape}")
print("\n--- VISTA PREVIA TRACKS ---")
display(tracks.head())

✅ Datos cargados correctamente.
   - Tracks: (100, 5)
   - Artists: (20, 4)

--- VISTA PREVIA TRACKS ---


,track_id,title,artist_id,popularity,duration_ms
0,TRK_000,Cancion_0,ART_11,22.0,154754
1,TRK_001,Cancion_1,ART_19,14.0,262483
2,TRK_002,Cancion_2,ART_02,42.0,253983
3,TRK_003,Cancion_3,ART_04,28.0,259752
4,TRK_004,Cancion_4,ART_18,35.0,204896


---
## 🧹 FASE 1: Limpieza (Cleaning)

Tenemos dos problemas en `tracks`:
1.  Hay `NaN` en `popularity`. Vamos a rellenarlos con la **media**.
2.  Hay tiempos negativos en `duration_ms`. Vamos a convertirlos a positivo (**valor absoluto**).

In [2]:
# 1. Rellenar Nulos (Imputación)
# Calcula la media de la columna popularity
media_pop = tracks['popularity'].mean()

# Rellena los huecos con esa media
tracks['popularity'] = tracks['popularity'].fillna(media_pop)

# 2. Corregir Errores
# Aplica el valor absoluto (.abs()) a la columna de duración
tracks['duration_ms'] = tracks['duration_ms'].abs()

print("Estado de limpieza:")
print(f"Nulos restantes: {tracks['popularity'].isnull().sum()}")
print(f"Valor mínimo duración: {tracks['duration_ms'].min()}")

Estado de limpieza:
Nulos restantes: 0
Valor mínimo duración: 50000


---
## 🔗 FASE 2: Fusión (Merging)

Queremos saber el **género** de cada canción, pero eso está en la tabla `artists`.
Une `tracks` (izquierda) con `artists` (derecha) usando la columna común `artist_id`.
*   Usa `how='left'` para no perder canciones.

In [3]:
# Sintaxis: pd.merge(izq, der, on='clave', how='tipo')

df_completo = pd.merge(
    tracks,
    artists,       # Tabla derecha
    on='artist_id',  # Clave común
    how='left'
)

print(f"Fusión completada. Dimensiones: {df_completo.shape}")
display(df_completo.head(2))

Fusión completada. Dimensiones: (100, 8)


,track_id,title,artist_id,popularity,duration_ms,name,genre,followers
0,TRK_000,Cancion_0,ART_11,22.0,154754,Artista_11,Hip-Hop,655811
1,TRK_001,Cancion_1,ART_19,14.0,262483,Artista_19,Latino,906778


---
## 📊 FASE 3: Agrupación (Grouping)

Pregunta de negocio: **¿Qué género musical es el más popular de media?**
1.  Agrupa por `genre`.
2.  Calcula la media (`mean`) de la columna `popularity`.

In [5]:
# Agrupa y calcula
ranking_generos = df_completo.groupby('genre')['popularity'].mean()

# Ordenamos para ver el ganador (descendente)
ranking_generos = ranking_generos.sort_values(ascending=False)

print("--- Ranking de Géneros ---")
display(ranking_generos)

--- Ranking de Géneros ---


,popularity
genre,
Jazz,63.275862
Latino,49.238095
Hip-Hop,42.906970
Rock,40.433684
Pop,39.000000


---
## 🧠 FASE 4: Transformación (Apply)

Vamos a categorizar las canciones.
*   Si popularidad >= 80 -> "Hit"
*   Si popularidad < 80 -> "Normal"

Usa una función **Lambda** dentro de `.apply()`.

In [6]:
# Sintaxis lambda: lambda x: "ValorSi" if Condicion else "ValorNo"

df_completo['categoria'] = df_completo['popularity'].apply(
    lambda x: "Hit" if x >= 80 else "Normal"
)

print("--- Categorización ---")
print(df_completo['categoria'].value_counts())

--- Categorización ---
categoria
Normal    82
Hit       18
Name: count, dtype: int64


---
## 🏁 VALIDACIÓN FINAL
Ejecuta esta celda para comprobar si tu entrenamiento ha sido exitoso.

In [7]:
# --- 🤖 CÓDIGO DE VALIDACIÓN (NO MODIFICAR) ---
def validar_entrenamiento():
    print("🚀 AUDITANDO ENTRENAMIENTO PANDAS...\n")
    puntos = 0
    errores = []

    # Variables
    v_tracks = globals().get('tracks')
    v_full = globals().get('df_completo')
    v_rank = globals().get('ranking_generos')

    # 1. LIMPIEZA
    if v_tracks['popularity'].isnull().sum() == 0:
        if v_tracks['duration_ms'].min() > 0:
            print("✅ [FASE 1] Limpieza: CORRECTO.")
            puntos += 2.5
        else:
            errores.append("❌ Aún hay duraciones negativas.")
    else:
        errores.append("❌ Aún hay nulos en popularidad.")

    # 2. MERGE
    if v_full is not None:
        if 'genre' in v_full.columns:
            if len(v_full) == 100:
                print("✅ [FASE 2] Merge: CORRECTO.")
                puntos += 2.5
            else:
                errores.append("❌ El tamaño del DataFrame ha cambiado incorrectamente tras el merge.")
        else:
            errores.append("❌ No se encuentra la columna 'genre' (fallo en el merge).")
    else:
        errores.append("❌ No existe 'df_completo'.")

    # 3. GROUPBY
    if v_rank is not None:
        if len(v_rank) > 0:
            print("✅ [FASE 3] Agrupación: CORRECTO.")
            puntos += 2.5
        else:
            errores.append("❌ El ranking está vacío.")

    # 4. APPLY
    if v_full is not None and 'categoria' in v_full.columns:
        if "Hit" in v_full['categoria'].values:
            print("✅ [FASE 4] Transformación Apply: CORRECTO.")
            puntos += 2.5
        else:
            errores.append("❌ La lógica del apply no parece correcta (no hay Hits).")
    else:
        errores.append("❌ No se encuentra la columna 'categoria'.")

    print("\n" + "="*50)
    if puntos == 10:
        print(f"🎉 ¡ENTRENAMIENTO COMPLETADO! Estás listo para el Hito 1.")
    else:
        print("⚠️ REVISA LOS ERRORES:")
        for e in errores: print(f"   - {e}")
    print("="*50)

validar_entrenamiento()

🚀 AUDITANDO ENTRENAMIENTO PANDAS...

✅ [FASE 1] Limpieza: CORRECTO.
✅ [FASE 2] Merge: CORRECTO.
✅ [FASE 3] Agrupación: CORRECTO.
✅ [FASE 4] Transformación Apply: CORRECTO.

🎉 ¡ENTRENAMIENTO COMPLETADO! Estás listo para el Hito 1.
